In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as trans
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time

%matplotlib inline

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
# Ouput size calcaltions wrto padding and kernel size
input_size = 28
kernel_size = 5
padding = 2
stride = 1
output_size = (input_size - kernel_size + 2*padding)/stride + 1

print(output_size)

# Solve for outpt_size=input_size
padding = (kernel_size - 1)/2

28.0


In [3]:
trainSet = dsets.MNIST(
    root='./data', 
    train=True, 
    transform=trans.ToTensor(), 
    download=True)

testSet = dsets.MNIST(root='./data', 
                      train = False, 
                      transform=trans.ToTensor(), 
                      download=True)

In [4]:
print('Number of images {}'.format(len(trainSet)))
print('Type {}'.format(type(trainSet[0][0])))
print('size of images {}'.format(trainSet[0][0].size()))

Number of images 60000
Type <class 'torch.Tensor'>
size of images torch.Size([1, 28, 28])


In [18]:
class Model4_6(nn.Module):
    def __init__(self):
        super(Model4_6, self).__init__() 
        self.cnn = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32*14*14, 10)
        
    def forward(self, x):
        out = self.cnn(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out=out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
class Model4_7(nn.Module):
    def __init__(self):
        super(Model4_7, self).__init__() 
        self.layer1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))        
        self.fc1 = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out= self.layer1(x)
        out= self.layer2(out)
        out=out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
class Model4_8(nn.Module):
    def __init__(self):
        super(Model4_8, self).__init__() 
        self.layer1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
             nn.BatchNorm2d(16),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
             nn.BatchNorm2d(32),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))        
        self.fc1 = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out= self.layer1(x)
        out= self.layer2(out)
        out=out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [19]:
def accuracy(testLoader, model):    
    correct, total = 0, 0
    with torch.no_grad():
        for data in testLoader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model.forward(images).to(device)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()      
    return(correct / total)


def benchmark(trainLoader, testLoader, model, epochs=1, lr=0.01):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    #lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda x: x*0.7)
    loss_function = nn.CrossEntropyLoss()
    
    start = time.time()
    for epoch in range(epochs):
        runningLoss = 0.0
        for i, (images, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            images = images.to(device)
            labels = labels.to(device)
            out = model(images).to(device)
            loss = loss_function(out, labels)
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
        print('epoch {}, loss {}'.format(epoch, runningLoss))
        #lr_scheduler.step()
        #if epoch%5 == 0:
        #    lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda x: x*0.7)

    print('Accuracy: {0:.4f}'.format(accuracy(testLoader, model)))
    print('Training time: {0:.2f}'.format(time.time() - start))    

In [20]:
lr = 0.05
batchSize = 100

trainloader = torch.utils.data.DataLoader(
    dataset=trainSet,
    batch_size=batchSize,
    num_workers = 12,
    pin_memory=True,
    shuffle=True)

testloader = torch.utils.data.DataLoader(dataset=testSet,  
    batch_size=100,
    num_workers = 12,
    pin_memory=True)

In [21]:
print('Conv:')
benchmark(trainloader, testloader, Model4_6(), epochs=5, lr = 0.1)

Conv:
epoch 0, loss 173.29868956655264
epoch 1, loss 67.37345357052982
epoch 2, loss 48.38780892081559
epoch 3, loss 39.65769081003964
epoch 4, loss 34.57854921929538
Accuracy: 0.9836
Training time: 9.27


In [13]:
print('Conv:')
benchmark(trainloader, testloader, Model4_7(), epochs=5, lr = 0.1)

Conv:
epoch 0, loss 163.62305738590658
epoch 1, loss 47.52382941870019
epoch 2, loss 35.05345236649737
epoch 3, loss 29.322012143675238
epoch 4, loss 25.167531778570265
Accuracy: 0.9870
Training time: 12.30


In [22]:
print('Conv:')
benchmark(trainloader, testloader, Model4_8(), epochs=5, lr = 0.1)

Conv:
epoch 0, loss 110.16549961548299
epoch 1, loss 33.8766860216856
epoch 2, loss 26.12414692598395
epoch 3, loss 21.233242574497126
epoch 4, loss 18.69966879801359
Accuracy: 0.9901
Training time: 13.84
